
## RL Cheat Sheet: Tipos de Acción y Algoritmos

| Tipo de Acción / PDF        | Algoritmo                  | Actualización Online / Fórmulas |
|------------------------------|---------------------------|--------------------------------|
| **Discreta (pocas acciones)** | Q-Learning (tabular / Deep Q) | **Simple:**<br>$$Q_k = Q_{k-1} + \alpha (r_k - Q_{k-1})$$<br>**Bellman / TD:**<br>$$Q(s,a) \gets Q(s,a) + \alpha [r + \gamma \max_{a'} Q(s',a') - Q(s,a)]$$ |
| **Discreta (política suave)** | Policy Gradient (Softmax) | **Acción tomada $i$:**<br>$$\theta_i \gets \theta_i + \alpha (1 - p_i)(r - b)$$<br>**Acciones no tomadas $j\neq i$:**<br>$$\theta_j \gets \theta_j - \alpha p_j (r - b)$$ |
| **Continua (acciones reales)** | Policy Gradient (Gaussian / REINFORCE) | **Media incremental:**<br>$$\hat{\mu}_{t+1} = \hat{\mu}_t + \frac{1}{N}(x_{t+1} - \hat{\mu}_t)$$<br>**Varianza incremental:**<br>$$\hat{\sigma}^2_{t+1} = \hat{\sigma}^2_t + \frac{(x_{t+1} - \hat{\mu}_t)^2 - \hat{\sigma}^2_t}{t+1}$$<br>o alternativamente:<br>$$\hat{\sigma}^2_{t+1} = \hat{\sigma}^2_t + \frac{1}{N}((x_t - \hat{\mu}) - \hat{\sigma}^2_t)$$ |

<br>
<br>

| Concepto | Fórmula / Descripción |
|----------|--------------------|
| **Media secuencial** | $$\hat{\mu}_{t+1} = \hat{\mu}_t + \frac{(x_{t+1} - \hat{\mu}_t)}{N}$$ |
| **Media normal (MLE)** | $$\hat{\mu}_{\text{MLE}} = \frac{1}{N} \sum_{i=1}^{N} x_i$$ |
| **Varianza secuencial** | $$\hat{\sigma}^2_{t+1} = \hat{\sigma}^2_t + \frac{(x_t - \hat{\mu})^2 - \hat{\sigma}^2_t}{N}$$ |
| **Varianza normal (MLE)** | $$\hat{\sigma}^2_{\text{MLE}} = \frac{1}{N} \sum_{i=1}^{N} (x_i - \hat{\mu})^2$$ |
| **Softmax** | $$\pi(a=j \mid s) = \frac{e^{\beta a_j}}{\sum_{i=1}^{k} e^{\beta a_i}}$$ |
| **Derivada softmax (acción elegida)** | $$\theta_i \gets \theta_i + \alpha (1 - p_i)(r - b)$$ $$\frac{\partial S_j}{\partial a_j} = S_j(1 - S_j)$$|
| **Derivada softmax (acción no elegida)** | $$\theta_j \gets \theta_j - \alpha p_j (r - b)$$ $$\frac{\partial S_j}{\partial a_i} = -S_j S_i$$ |
| **Actualización parámetros (datos discretos)** | $$a_j \gets a_j + \alpha (1 - s_j) R$$ <br> $$a_i \gets a_i + \alpha (0 - s_i) R, \quad i \neq j$$ <br> $s_j$ = probabilidad actual de la acción $j$ |
| **Q-Learning** | $$Q(s,a) \gets Q(s,a) + \alpha [r + \gamma \max_{a'} Q(s',a') - Q(s,a)]$$ $\gamma$: factor de descuento → cuánto valen las recompensas futuras. <br> - Si $\gamma \approx 1$: agente paciente, valora recompensas futuras. <br> - Si $\gamma$ es bajo: agente solo considera recompensas inmediatas. |
| **Decaimiento exponencial de ε** | $$\text{eps\_threshold} = \text{eps\_end} + (\text{eps\_start} - \text{eps\_end}) \cdot \exp(- \text{steps} / \text{eps\_decay})$$ <br> - Con probabilidad $1-\epsilon$: explota → elige acción con mayor $Q$. <br> - Con probabilidad $\epsilon$: explora → elige acción aleatoria. |
| **PDF de distribución normal** | $$f(x) = \frac{1}{\sigma \sqrt{2\pi}} e^{- \frac{(x - \mu)^2}{2\sigma^2}}$$ |
| **Valor esperado** | $$E[f(X)] = \sum p(x_i)\, f(x_i)$$ $$Q_k = Q_{k-1} + \frac{1}{k}(r_k - Q_{k-1})$$ |

<br>

## Ejemplo: Actualización de parámetros softmax (categorica) (Policy Gradient online)

Condiciones iniciales:

- Parámetros iniciales: 
$$
\theta = [0, 0]
$$
- Softmax inicial: 
$$
\text{softmax}(\theta) = [0.5, 0.5]
$$
- Learning rate: $\alpha = 0.1$  
- Recompensa: $r = 1$  
- Acción seleccionada: índice 0  

Fórmula de actualización:

$$
\theta \gets \theta + \alpha \cdot (\text{grad} - \text{probs}) \cdot r
$$

donde para la acción seleccionada:  
$$
\text{grad} = [1,0]
$$

---

### Iteraciones (resumen)

| Iteración | $\theta_0$ | $\theta_1$ | Softmax $\text{probs}$ |
|-----------|------------|------------|-----------------------|
| 0         | 0          | 0          | [0.5, 0.5]           |
| 1         | 0.05       | -0.05      | [0.512, 0.488]       |
| 2         | 0.0975     | -0.0975    | [0.524, 0.476]       |
| 3         | 0.1426     | -0.1426    | [0.537, 0.463]       |
| 4         | 0.1891     | -0.1891    | [0.549, 0.451]       |
| 5         | 0.2344     | -0.2344    | [0.562, 0.438]       |

- Los parámetros de la **acción elegida aumentan**, los de la **no elegida disminuyen**.  
- Las probabilidades softmax favorecen progresivamente la acción seleccionada.  

---


## Ejemplo: Actualización de parámetros con Policy Gradient (Continua) (Gaussian PDF, Online)

### Condiciones iniciales

- Distribución de acciones: $a \sim \mathcal{N}(\mu, \sigma^2)$  
- Parámetros iniciales: 
$$
\mu_0 = 0, \quad \sigma_0^{2} = 1
$$
- Learning rate implícito mediante promedio online: $\frac{1}{N}$  
- Recompensas asumidas $r_t = 1$  
- Acciones tomadas por iteración: $a = [0.5, 0.6, 0.4, 0.7, 0.3]$

---

### Fórmulas de actualización online

- Media incremental:
$$
\hat{\mu}_{t+1} = \hat{\mu}_t + \frac{1}{t+1} \left(a_{t+1} - \hat{\mu}_t\right)
$$

- Varianza incremental:
$$
\hat{\sigma}^2_{t+1} = \hat{\sigma}^2_t + \frac{(a_{t+1} - \hat{\mu}_t)^2 - \hat{\sigma}^2_t}{t+1}
$$

> Nota: Esta es la forma **online**, equivalente a un promedio ponderado secuencial que se puede usar en Policy Gradient.

---

### Iteración 0

- Acción: $a_1 = 0.5$  
- Media:
$$
\mu_1 = 0 + \frac{0.5 - 0}{1} = 0.5
$$
- Varianza:
$$
\sigma_1^2 = 1 + \frac{(0.5 - 0)^2 - 1}{1} = 0.25
$$

---

### Iteración 1

- Acción: $a_2 = 0.6$  
- Media:
$$
\mu_2 = 0.5 + \frac{0.6 - 0.5}{2} = 0.55
$$
- Varianza:
$$
\sigma_2^2 = 0.25 + \frac{(0.6 - 0.5)^2 - 0.25}{2} = 0.125
$$

---

### Iteración 2

- Acción: $a_3 = 0.4$  
- Media:
$$
\mu_3 = 0.55 + \frac{0.4 - 0.55}{3} \approx 0.5
$$
- Varianza:
$$
\sigma_3^2 = 0.125 + \frac{(0.4 - 0.55)^2 - 0.125}{3} \approx 0.0833
$$

---

### Iteración 3

- Acción: $a_4 = 0.7$  
- Media:
$$
\mu_4 = 0.5 + \frac{0.7 - 0.5}{4} = 0.55
$$
- Varianza:
$$
\sigma_4^2 = 0.0833 + \frac{(0.7 - 0.5)^2 - 0.0833}{4} \approx 0.0917
$$

---

### Iteración 4

- Acción: $a_5 = 0.3$  
- Media:
$$
\mu_5 = 0.55 + \frac{0.3 - 0.55}{5} = 0.5
$$
- Varianza:
$$
\sigma_5^2 = 0.0917 + \frac{(0.3 - 0.55)^2 - 0.0917}{5} \approx 0.0834
$$

---

### Resumen de iteraciones

| Iteración | Acción $a_t$ | $\mu_t$ | $\sigma^2_t$ |
|-----------|--------------|---------|-------------|
| 0         | 0.5          | 0.5     | 0.25        |
| 1         | 0.6          | 0.55    | 0.125       |
| 2         | 0.4          | 0.5     | 0.0833      |
| 3         | 0.7          | 0.55    | 0.0917      |
| 4         | 0.3          | 0.5     | 0.0834      |

- La **media $\mu$** se ajusta hacia las acciones tomadas con mayor recompensa.  
- La **varianza $\sigma^2$** refleja la dispersión de las acciones y se ajusta secuencialmente en cada iteración.  
- Este enfoque es equivalente a una actualización online de Policy Gradient para distribuciones gaussianas.


---

## Ejemplo: Actualización de Q-Learning (Tabular)

##### Condiciones iniciales

- Estado único: $s$  
- Acciones posibles: $A = \{a_0, a_1\}$  
- Valores iniciales: 
$$
Q(s, a_0) = 0, \quad Q(s, a_1) = 0
$$
- Learning rate: $\alpha = 0.1$  
- Recompensas recibidas por acción: $r = [1, 0.5, 1, 0, 0.7]$  
- No se considera futuro (TD simple, $\gamma = 0$)  

##### Fórmula de actualización simple (Tabular Q-Learning)

$$
Q_k = Q_{k-1} + \alpha \cdot (r_k - Q_{k-1})
$$

> Nota: El nuevo Q es el viejo Q más una fracción del error (diferencia entre recompensa recibida y la predicción previa).

---

##### Iteraciones

| Iteración | Acción tomada | $Q_{prev}$ | Recompensa $r$ | $Q_{new}$ |
|-----------|---------------|------------|----------------|-----------|
| 0         | $a_0$         | 0          | 1              | $0 + 0.1*(1-0) = 0.1$ |
| 1         | $a_1$         | 0          | 0.5            | $0 + 0.1*(0.5-0) = 0.05$ |
| 2         | $a_0$         | 0.1        | 1              | $0.1 + 0.1*(1-0.1) = 0.19$ |
| 3         | $a_1$         | 0.05       | 0              | $0.05 + 0.1*(0-0.05) = 0.045$ |
| 4         | $a_0$         | 0.19       | 0.7            | $0.19 + 0.1*(0.7-0.19) = 0.241$ |

---

##### Resumen

- Los valores $Q(s, a)$ se **acercan progresivamente a la recompensa promedio** de cada acción.  
- La acción $a_0$ tiene un Q mayor porque históricamente dio más recompensa.  
- La **fórmula es muy intuitiva:** ajusta el valor anterior usando el error entre lo que se recibió y lo esperado.

